In [1]:
import os
import time
import numpy as np
import pandas as pd
import random
import math

from copy import deepcopy
from six import add_metaclass
from abc import ABCMeta, abstractmethod
from scipy import optimize
from deap.benchmarks import schwefel

import matplotlib.pyplot as plt

import folium

In [2]:
trajectory1 = np.load( os.getcwd() + '/SampleTraj/Trj_0.npy' )
trajectory1 = pd.DataFrame(trajectory1)
trajectory1.columns = ['timestamp', 'latitude', 'longitude', 'altitude']
trajectory1 = trajectory1.drop_duplicates(subset=['latitude', 'longitude'], keep='first').to_numpy()


trajectory2 = np.load( os.getcwd() + '/SampleTraj/Trj_1.npy' )
trajectory2 = pd.DataFrame(trajectory2)
trajectory2.columns = ['timestamp', 'latitude', 'longitude', 'altitude']
trajectory2 = trajectory2.drop_duplicates(subset=['latitude', 'longitude'], keep='first').to_numpy()


trajectory3 = np.load( os.getcwd() + '/SampleTraj/Trj_2.npy' )
trajectory3 = pd.DataFrame(trajectory3)
trajectory3.columns = ['timestamp', 'latitude', 'longitude', 'altitude']
trajectory3 = trajectory3.drop_duplicates(subset=['latitude', 'longitude'], keep='first').to_numpy()

trajectory4 = np.load( os.getcwd() + '/SampleTraj/Trj_3.npy' )
trajectory4 = pd.DataFrame(trajectory4)
trajectory4.columns = ['timestamp', 'latitude', 'longitude', 'altitude']
trajectory4 = trajectory4.drop_duplicates(subset=['latitude', 'longitude'], keep='first').to_numpy()
# targetPoint = -88

In [3]:
def aux_CalculateTrajectoryError( trajectory, variablePoints ):
    
    totalError = 0.0
    
    for k in range( np.size( variablePoints ) ):
        
        if variablePoints[ k ]:
            single_point_error = aux_CalculateErrorSinglePoint( trajectory, k )
            
            if ~np.isnan(single_point_error):

                totalError += single_point_error
                
    
    return totalError




def aux_CalculateErrorSinglePoint( trajectory, targetPoint ):
    
    
    v_m1 = aux_haversine( trajectory[ targetPoint - 1, 2 ], trajectory[ targetPoint - 1, 1 ], trajectory[ targetPoint, 2 ], trajectory[ targetPoint, 1 ] )
    s_m1 = v_m1 / ( trajectory[ targetPoint, 0 ] - trajectory[ targetPoint - 1, 0 ] )
    
    
    v_m2 = aux_haversine( trajectory[ targetPoint - 2, 2 ], trajectory[ targetPoint - 2, 1 ], trajectory[ targetPoint - 1, 2 ], trajectory[ targetPoint - 1, 1 ] )
    s_m2 = v_m2 / ( trajectory[ targetPoint - 1, 0 ] - trajectory[ targetPoint - 2, 0 ] )
    
    
    v_p1 = aux_haversine( trajectory[ targetPoint, 2 ], trajectory[ targetPoint, 1 ], trajectory[ targetPoint + 1, 2 ], trajectory[ targetPoint + 1, 1 ] )
    s_p1 = v_p1 / ( trajectory[ targetPoint + 1, 0 ] - trajectory[ targetPoint, 0 ] )
    
    
    v_p2 = aux_haversine( trajectory[ targetPoint + 1, 2 ], trajectory[ targetPoint + 1, 1 ], trajectory[ targetPoint + 2, 2 ], trajectory[ targetPoint + 2, 1 ] )
    s_p2 = v_p2 / ( trajectory[ targetPoint + 2, 0 ] - trajectory[ targetPoint + 1, 0 ] )
    

    return np.std( [ v_m2, v_m1, v_p1, v_p2 ] ) / np.mean( [ v_m2, v_m1, v_p1, v_p2 ] )



def aux_haversine(lon1, lat1, lon2, lat2): # haversine is the angle distance between two points on a sphere 
    
    from math import radians, cos, sin, asin, sqrt

    R = 6372.8

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

In [4]:
# @add_metaclass(ABCMeta)
# class ObjectiveFunction(object):

#     def __init__(self, name, dim, minf, maxf):
#         self.name = name
#         self.dim = dim
#         self.minf = minf
#         self.maxf = maxf

#     def sample(self):
#         return np.random.uniform(low=self.minf, high=self.maxf, size=self.dim)

#     def custom_sample(self):
#         return np.repeat(self.minf, repeats=self.dim) \
#                + np.random.uniform(low=0, high=1, size=self.dim) *\
#                np.repeat(self.maxf - self.minf, repeats=self.dim)

#     @abstractmethod
#     def evaluate(self, x):
#         pass


# class Sphere(ObjectiveFunction):

#     def __init__(self, dim):
#         super(Sphere, self).__init__('Sphere', dim, -100.0, 100.0)

#     def evaluate(self, x):
#         return sum(np.power(x, 2))


# class Rosenbrock(ObjectiveFunction):

#     def __init__(self, dim):
#         super(Rosenbrock, self).__init__('Rosenbrock', dim, -30.0, 30.0)

#     def evaluate(self, x):
#         return optimize.rosen(x)


# class Rastrigin(ObjectiveFunction):

#     def __init__(self, dim):
#         super(Rastrigin, self).__init__('Rastrigin', dim, -5.12, 5.12)

#     def evaluate(self, x):
#         return 10 * len(x)\
#                + np.sum(np.power(x, 2) - 10 * np.cos(2 * np.pi * np.array(x)))


# class Schwefel(ObjectiveFunction):

#     def __init__(self, dim):
#         super(Schwefel, self).__init__('Schwefel', dim, -500.0, 500.0)

#     def evaluate(self, x):
#         return schwefel(x)[0]

# Artificial bee colony

In [22]:
@add_metaclass(ABCMeta)
class ArtificialBee:
    
    TRIAL_INITIAL_DEFAULT_VALUE = 0
    INITIAL_DEFAULT_PROBABILITY = 0.0
    
    def __init__(self, trajectory, variablePoints, k, loss_function):
        
        self.trajectory = np.copy( trajectory )
        self.variablePoints = variablePoints
        self.k = k
        self.pos = self.trajectory[self.k]
        self.pos[1] += np.random.normal(0, .0005)
        self.pos[2] += np.random.normal(0, .0005)
        
        self.loss_function = loss_function

        self.fitness = loss_function(self.trajectory, self.variablePoints)
        self.trial = ArtificialBee.TRIAL_INITIAL_DEFAULT_VALUE
        self.prob = ArtificialBee.INITIAL_DEFAULT_PROBABILITY
        
    
    def update_bee(self, pos, fitness):
        
        if fitness <= self.fitness:
            
            self.pos = pos
            self.trajectory[self.k] = self.pos
            self.fitness = fitness
            self.trial = 0
            
        else:
            self.trial += 1
            
    def reset_bee(self, max_trials):
        
        if self.trial >= max_trials:
            
            self.__reset_bee()
            
    def __reset_bee(self):
        
        self.pos = self.trajectory[k]
        self.pos[1] += np.random.normal(0, .0005)
        self.pos[2] += np.random.normal(0, .0005)
        self.fitness = self.loss_function(self.trajectory, self.variablePoints)
        self.trial = ArtificialBee.TRIAL_INITIAL_DEFAULT_VALUE
        self.prob = ArtificialBee.INITIAL_DEFAULT_PROBABILITY

In [6]:
class EmployeeBee(ArtificialBee):
    
    def explore(self, max_trials):
        
        if self.trial <= max_trials:
            
            component = np.copy(self.pos)
            component[1] += np.random.normal(0, .001)
            component[2] += np.random.normal(0, .001)
            phi = np.random.uniform(low=-1, high=1, size=len(self.pos))
            n_pos = self.pos + (self.pos - component) * phi
            tmp = np.copy(self.trajectory)
            tmp[self.k] = n_pos
            n_fitness = self.loss_function(tmp, self.variablePoints)
            self.update_bee(n_pos, n_fitness)
            
    def get_fitness(self):
        return 1 / (1 + self.fitness) if self.fitness >= 0 else 1 + np.abs(self.fitness)
    
    def compute_prob(self, max_fitness):
        self.prob = self.get_fitness / max_fitness

In [7]:
class OnlookerBee(ArtificialBee):
    
    def onlook(self, best_food_sources, max_trials):
        candidate = np.random.choice(best_food_sources)
        self.__exploit(candidate.pos, candidate.fitness, max_trials)
    
    
    def __exploit(self, candidate, fitness, max_trials):
        
        if self.trial <= max_trials:
            
            component = np.random.choice(candidate)
            phi = np.random.uniform(low=-1, high=1, size=len(component))
            n_pos = candidate + (candidate - component) * phi
#             n_pos = self.evaluate_boundries(n_pos)
            tmp = np.copy(self.trajectory)
            tmp[self.k] = n_pos
            n_fitness = self.loss_function(tmp, self.variablePoints)
            
            if n_fitness <= fitness:
                
                self.pos = n_pos
                self.trajectory[self.k] = self.pos
                self.fitness = n_fitness
                self.trial = 0
                
            else:
                
                self.trial += 1

In [8]:
class ABC:
    
    def __init__(self, trajectory, variablePoints, loss_function, colony_size, n_iter, max_trials):
        
        self.trajectory = np.copy(trajectory)
        self.variablePoints = variablePoints
        self.colony_size = colony_size
        self.loss_function = loss_function
        self.n_iter = n_iter
        self.max_trials = max_trials
        
        self.optimal_solution = dict()
        self.optimality_tracking = dict()
        
        for k in np.where(self.variablePoints)[0]:
            self.optimal_solution[k] = None
            
            self.optimality_tracking[k] = np.copy(self.trajectory[k])
        
        
        self.employee_bees = dict()
        self.onlooker_bees = dict()
        
    
    def __reset_algorithm(self):
        self.optimal_solution = dict()
        self.optimality_tracking = dict()
        
        for k in np.where(self.variablePoints)[0]:
            self.optimal_solution[k] = None
            
            self.optimality_tracking[k] = np.copy(self.trajectory[k])
        
        
        
    def __update_optimality_tracking(self, k):

        Error = aux_CalculateTrajectoryError(self.trajectory, self.variablePoints)
        tmp = np.copy(self.trajectory)
        tmp[k] = self.optimal_solution[k].pos
        error = aux_CalculateTrajectoryError(tmp, self.variablePoints)
        
        if error < Error:
            
            print(f'For point {k} in trajectory.')
            print(f'Old error: {Error}')
            print(f'New error: {error}')
            print()
            
            self.trajectory = np.copy(tmp)
            
            self.optimality_tracking[k] = np.copy( self.optimal_solution[k].pos )
        
        
    def __update_optimal_solution(self, k):
        n_optimal_solution = min(self.onlooker_bees[k] + self.employee_bees[k], key=lambda bee: bee.fitness)
        
        if not self.optimal_solution[k]:
            
            self.optimal_solution[k] = deepcopy(n_optimal_solution)
        
        else:
            
            if n_optimal_solution.fitness < self.optimal_solution[k].fitness:
                
                print(f'New optimal solution for {k}: {n_optimal_solution.pos}')
                              
                self.optimal_solution[k] = deepcopy(n_optimal_solution)
    
    
    
    def __initialize_employees(self, k):

        self.employee_bees[k] = []
        
        for _ in range(self.colony_size // 2):
            self.employee_bees[k].append(EmployeeBee(self.trajectory, self.variablePoints, k, self.loss_function))
            
    def __initialize_onlookers(self, k):

        self.onlooker_bees[k] = []
        
        for _ in range(self.colony_size // 2):
            self.onlooker_bees[k].append(OnlookerBee(self.trajectory, self.variablePoints, k, self.loss_function))
            
    def __employee_bees_phase(self, k):
        map(lambda bee: bee.explore(self.max_trials), self.employee_bees[k])
        
    def __calculate_probabilities(self, k):
        
        sum_fitness = sum(map(lambda bee: bee.get_fitness(), self.employee_bees[k]))
        
        map(lambda bee: bee.compute_prob(sum_fitness), self.employee_bees[k])
        
    
    def __select_best_food_sources(self, k):
        
        self.best_food_sources = filter(lambda bee: bee.prob > np.random.uniform(low=0, high=1), self.employee_bees[k])
        
        while not self.best_food_sources:
            
            self.best_food_sources = filter(lambda bee: bee.prob > np.random.uniform(low=0, high=1), self.employee_bees[k])
            
    def __onlooker_bees_phase(self, k):
        map(lambda bee: bee.onlook(self.best_food_sources, self.max_trials), self.onlooker_bees[k])
        
    def __scout_bees_phase(self, k):
        map(lambda bee: bee.reset_bee(self.max_trials), self.onlooker_bees[k] + self.employee_bees[k])
        
    def optimize(self):
        
        self.__reset_algorithm()
        
        for k in np.where(self.variablePoints)[0]:

            self.__initialize_employees(k)
            self.__initialize_onlookers(k)
        
        
        for i in range(self.n_iter):
            
            for k in np.where(self.variablePoints)[0]:
            
                self.__employee_bees_phase(k)
                self.__update_optimal_solution(k)

                self.__calculate_probabilities(k)
                self.__select_best_food_sources(k)

                self.__onlooker_bees_phase(k)
                self.__scout_bees_phase(k)

                self.__update_optimal_solution(k)
                self.__update_optimality_tracking(k)
            

            

In [9]:
def simulate(trajectory, variablePoints, loss_function, colony_size=300, n_iter=1000, max_trials=100, simulations=200):
    
    bestTraj = np.copy(trajectory)
    
    for s in range(simulations):
        
        print(f'Simulation: {s}')
        
        optimizer = ABC(bestTraj, variablePoints, loss_function, colony_size, n_iter, max_trials)
        
        optimizer.optimize()
        
        
        for k in np.where(variablePoints)[0]:
            bestTraj[k] = optimizer.optimality_tracking[k]
            

            
    return bestTraj

In [10]:
targetPoints = []

targetPoints.append(np.array(range(299, 302)))

targetPoints = np.hstack(targetPoints)

In [11]:
variablePoints = np.zeros( ( np.size( trajectory1, 0 ) ), dtype = bool )

In [12]:
for i in range(len(variablePoints)):
    if i in targetPoints:
        variablePoints[i] = True

In [13]:
start_time = time.time()

correctedTraj1 = simulate(trajectory1, variablePoints, aux_CalculateTrajectoryError)

exec_time1 = time.time() - start_time

Simulation: 0
For point 299 in trajectory.
Old error: 2.467576363506674
New error: 2.4581287487797594

For point 300 in trajectory.
Old error: 2.4581287487797594
New error: 2.4574389759337087

For point 301 in trajectory.
Old error: 2.4574389759337087
New error: 2.442911361894864

Simulation: 1
For point 299 in trajectory.
Old error: 2.442911361894864
New error: 2.4353817426206437

For point 300 in trajectory.
Old error: 2.4353817426206437
New error: 2.4345818153498406

For point 301 in trajectory.
Old error: 2.4345818153498406
New error: 2.4223287450846276

Simulation: 2
For point 299 in trajectory.
Old error: 2.4223287450846276
New error: 2.411765842743105

For point 300 in trajectory.
Old error: 2.411765842743105
New error: 2.411232330827058

For point 301 in trajectory.
Old error: 2.411232330827058
New error: 2.3993002986565797

Simulation: 3
For point 299 in trajectory.
Old error: 2.3993002986565797
New error: 2.3909734783309258

For point 300 in trajectory.
Old error: 2.390973478

Simulation: 30
For point 299 in trajectory.
Old error: 1.934485018367909
New error: 1.928145584580418

For point 300 in trajectory.
Old error: 1.928145584580418
New error: 1.9270114889394292

For point 301 in trajectory.
Old error: 1.9270114889394292
New error: 1.9192404372908787

Simulation: 31
For point 299 in trajectory.
Old error: 1.9192404372908787
New error: 1.913622958165122

For point 300 in trajectory.
Old error: 1.913622958165122
New error: 1.9123148527498184

For point 301 in trajectory.
Old error: 1.9123148527498184
New error: 1.9031551779635785

Simulation: 32
For point 299 in trajectory.
Old error: 1.9031551779635785
New error: 1.8960631739830012

For point 300 in trajectory.
Old error: 1.8960631739830012
New error: 1.8948505372435835

For point 301 in trajectory.
Old error: 1.8948505372435835
New error: 1.8878291920244807

Simulation: 33
For point 299 in trajectory.
Old error: 1.8878291920244807
New error: 1.8816619733903472

For point 300 in trajectory.
Old error: 1.881

Simulation: 59
For point 299 in trajectory.
Old error: 1.5150374012327257
New error: 1.5101910465105304

For point 300 in trajectory.
Old error: 1.5101910465105304
New error: 1.50856856200229

For point 301 in trajectory.
Old error: 1.50856856200229
New error: 1.502889400913592

Simulation: 60
For point 299 in trajectory.
Old error: 1.502889400913592
New error: 1.497826384972997

For point 300 in trajectory.
Old error: 1.497826384972997
New error: 1.4964180895619648

For point 301 in trajectory.
Old error: 1.4964180895619648
New error: 1.4905311799928693

Simulation: 61
For point 299 in trajectory.
Old error: 1.4905311799928693
New error: 1.4858847748020891

For point 300 in trajectory.
Old error: 1.4858847748020891
New error: 1.4842439160770553

For point 301 in trajectory.
Old error: 1.4842439160770553
New error: 1.4772415884043593

Simulation: 62
For point 299 in trajectory.
Old error: 1.4772415884043593
New error: 1.471600749366788

For point 300 in trajectory.
Old error: 1.4716007

Simulation: 88
For point 299 in trajectory.
Old error: 1.1946951593995652
New error: 1.191014970068757

For point 300 in trajectory.
Old error: 1.191014970068757
New error: 1.189071824704461

For point 301 in trajectory.
Old error: 1.189071824704461
New error: 1.1848958834639451

Simulation: 89
For point 299 in trajectory.
Old error: 1.1848958834639451
New error: 1.182017287465295

For point 300 in trajectory.
Old error: 1.182017287465295
New error: 1.1801979151139599

For point 301 in trajectory.
Old error: 1.1801979151139599
New error: 1.1761321883678828

Simulation: 90
For point 299 in trajectory.
Old error: 1.1761321883678828
New error: 1.1732597553947768

For point 300 in trajectory.
Old error: 1.1732597553947768
New error: 1.1717704242408442

For point 301 in trajectory.
Old error: 1.1717704242408442
New error: 1.1679793832849097

Simulation: 91
For point 299 in trajectory.
Old error: 1.1679793832849097
New error: 1.1650722230126365

For point 300 in trajectory.
Old error: 1.1650

For point 299 in trajectory.
Old error: 0.9850047704715086
New error: 0.9831340854457924

For point 300 in trajectory.
Old error: 0.9831340854457924
New error: 0.9822190628459728

For point 301 in trajectory.
Old error: 0.9822190628459728
New error: 0.9793916247706074

Simulation: 118
For point 299 in trajectory.
Old error: 0.9793916247706074
New error: 0.9771678561539997

For point 300 in trajectory.
Old error: 0.9771678561539997
New error: 0.976216885804023

For point 301 in trajectory.
Old error: 0.976216885804023
New error: 0.9730769495209906

Simulation: 119
For point 299 in trajectory.
Old error: 0.9730769495209906
New error: 0.9713816037804496

For point 300 in trajectory.
Old error: 0.9713816037804496
New error: 0.9705699981052216

For point 301 in trajectory.
Old error: 0.9705699981052216
New error: 0.9680174076535866

Simulation: 120
For point 299 in trajectory.
Old error: 0.9680174076535866
New error: 0.9663507769738751

For point 300 in trajectory.
Old error: 0.966350776973

Simulation: 146
For point 299 in trajectory.
Old error: 0.8600074427114163
New error: 0.8595460390387192

For point 300 in trajectory.
Old error: 0.8595460390387192
New error: 0.8594618343390381

For point 301 in trajectory.
Old error: 0.8594618343390381
New error: 0.8561157601719529

Simulation: 147
For point 299 in trajectory.
Old error: 0.8561157601719529
New error: 0.8558503917731533

For point 301 in trajectory.
Old error: 0.8558503917731533
New error: 0.8527523252125042

For point 300 in trajectory.
Old error: 0.8527523252125042
New error: 0.8526716529693167

Simulation: 148
For point 299 in trajectory.
Old error: 0.8526716529693167
New error: 0.8525157887787344

For point 300 in trajectory.
Old error: 0.8525157887787344
New error: 0.8524788045665044

For point 301 in trajectory.
Old error: 0.8524788045665044
New error: 0.8495950554155072

Simulation: 149
For point 299 in trajectory.
Old error: 0.8495950554155072
New error: 0.8494144374304067

For point 300 in trajectory.
Old err

For point 299 in trajectory.
Old error: 0.7798324506911086
New error: 0.779771677372006

For point 301 in trajectory.
Old error: 0.779771677372006
New error: 0.7776386564232878

For point 300 in trajectory.
Old error: 0.7776386564232878
New error: 0.7776147014354124

Simulation: 176
For point 299 in trajectory.
Old error: 0.7776147014354124
New error: 0.777552041824034

For point 301 in trajectory.
Old error: 0.777552041824034
New error: 0.7746766635573559

For point 300 in trajectory.
Old error: 0.7746766635573559
New error: 0.774644756733858

Simulation: 177
For point 299 in trajectory.
Old error: 0.774644756733858
New error: 0.7744458076495488

For point 301 in trajectory.
Old error: 0.7744458076495488
New error: 0.7719352471385128

For point 300 in trajectory.
Old error: 0.7719352471385128
New error: 0.7718600133743895

Simulation: 178
For point 299 in trajectory.
Old error: 0.7718600133743895
New error: 0.7718248913094361

For point 300 in trajectory.
Old error: 0.7718248913094361

In [14]:
print(f'Execution time of first trajectory: {exec_time1}')

Execution time of first trajectory: 498.3810706138611


In [15]:
tmp = trajectory1[:, 1:3]

In [16]:
cTraj1 = correctedTraj1[:, 1:3]

In [17]:
avg_lat = np.mean([row[0] for row in tmp])
avg_lon = np.mean([row[1] for row in tmp])

In [18]:
m = folium.Map(
    location=[avg_lat, avg_lon],
    tiles="cartodbpositron",
    zoom_start=5
)

aline1 = folium.PolyLine(tmp, color='blue')
m.add_child(aline1)

aline2 = folium.PolyLine(cTraj1, color='red')
m.add_child(aline2)

# aline3 = folium.PolyLine(trajectory1[299:301, 1:3], color='green')
# m.add_child(aline3)

m

In [19]:
aux_CalculateTrajectoryError(trajectory1, variablePoints)

2.467576363506674

In [20]:
aux_CalculateTrajectoryError(correctedTraj1, variablePoints)

0.7411546134172764

In [21]:
trajectory1[0]

array([1.6495488e+09, 4.2455000e+01, 1.4142800e+02, 6.1036200e+03])